In [1]:
!git clone https://github.com/dungwoong/CSC413Final.git
%cd CSC413Final

Cloning into 'CSC413Final'...
remote: Enumerating objects: 170, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 170 (delta 74), reused 126 (delta 34), pack-reused 0
Receiving objects: 100% (170/170), 2.48 MiB | 9.00 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/CSC413Final


In [1]:
# %cd "C:\Users\cyuan\Documents\CSC413FinalProject\CSC413Final\CSC413Final"

C:\Users\cyuan\Documents\CSC413FinalProject\CSC413Final\CSC413Final


In [11]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from math import ceil
from torch.utils.benchmark import Timer
import time
import os
import csv

In [3]:
from shufflenet_alt import ShuffleNetV2, ShuffleNetSE, ShuffleNetSLE, init_params

In [18]:
def save_to_csv(file_path, model_name, batch_size, avg_inference_time, avg_batch_time, images_per_second, batches_per_second):
    file_exists = os.path.isfile(file_path)
    
    with open(file_path, newline='', mode='a') as file:
        csv_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        if not file_exists:
            csv_writer.writerow(["Model", "Batch Size", "Average Inference Time(Image)", "Average Inference Time(Batch)", "Images/s", "Batches/s"])
        
        csv_writer.writerow([model_name, batch_size, avg_inference_time, avg_batch_time, images_per_second, batches_per_second])

In [20]:
# Change batch size here
batch_size_list = [32, 64, 128]

model = ShuffleNetSE(net_size=1)

# Change model path here
model_state_dict = torch.load('se1/tmp/0099.pth')['mod']

model.load_state_dict(model_state_dict)

# Set the model to evaluation mode and move it to the CUDA device
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
print(model.__class__)

mean, std = (0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)

# Define the transform to apply to the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Create the test dataset
test_set = datasets.CIFAR10(root="data", train=False, download=True, transform=transform)
num_images = len(test_set)

for batch_size in batch_size_list:
    print(f"Batch size: {batch_size}")

    # Create a DataLoader to generate batches of test data
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

    # Warm up the model by running it a few times
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(test_loader):
            if i == 5:
                break
            inputs = inputs.to(device)
            outputs = model(inputs)


    total_elapsed_time = 0.0
    total_samples = 0
    total_batches = 0

    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)

            start_time = time.time()
            outputs = model(inputs)
            elapsed_time = time.time() - start_time

            total_elapsed_time += elapsed_time
            total_samples += inputs.size(0)
            total_batches += 1

    # Calculate the average inference time per image
    images_per_second = total_samples/total_elapsed_time
    batches_per_second = total_batches/total_elapsed_time
    avg_inference_time = total_elapsed_time / total_samples
    avg_batch_time = total_elapsed_time / total_batches
    print(f"Average inference time per image: {avg_inference_time:.6f} seconds")
    print(f"Average inference time per batch: {avg_batch_time:.6f} seconds")
    print(f"Number of images per second: {images_per_second:.2f}")
    print(f"Number of batches per second: {batches_per_second:.2f}")
    save_to_csv('inference_time.csv', model.__class__, batch_size, avg_inference_time, avg_batch_time, images_per_second, batches_per_second)

cuda
<class 'shufflenet_alt.ShuffleNetSE'>
Files already downloaded and verified
Batch size: 32
Average inference time per image: 0.000302 seconds
Average inference time per batch: 0.009664 seconds
Number of images per second: 3306.02
Number of batches per second: 103.48
Batch size: 64
Average inference time per image: 0.000157 seconds
Average inference time per batch: 0.009991 seconds
Number of images per second: 6375.34
Number of batches per second: 100.09
Batch size: 128
Average inference time per image: 0.000076 seconds
Average inference time per batch: 0.009637 seconds
Number of images per second: 13135.45
Number of batches per second: 103.77
